In [ ]:
# Initiate GPU runtime

import tensorflow as tf
tf.test.gpu_device_name()

import torch
torch.cuda.get_device_name(0)

!rm -rf sample_data

In [ ]:
# Install Dorado (Oxford Nanopore Technologies)

%%shell

DORADO_VERSION=0.6.0 # Updated 02 Apr 2024

pip install pod5

wget https://cdn.oxfordnanoportal.com/software/analysis/dorado-"$DORADO_VERSION"-linux-x64.tar.gz -O dorado.tar.gz
tar -vzxf dorado.tar.gz; rm -rf dorado.tar.gz; mv dorado-"$DORADO_VERSION"-linux-x64 ont-dorado
./ont-dorado/bin/dorado download --directory ont-dorado/models
./ont-dorado/bin/dorado --version

In [ ]:
# Import Google Drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ls gdrive/MyDrive

In [ ]:
# Basecall and Demultiplex process

%%shell

# Specify the name of the folder (i.e., 20240411_1608_MN22990_FAT84733_f175dd69/fast5)
DIR="20240411_1608_MN22990_FAT84733_f175dd69/fast5"

# Specify the barcode kit used for library preparation
BARKIT="EXP-NBD114"

PREFIX="$(echo "$IDIR" | awk -F/ '{print $1}')"

pod5 convert fast5 $IDIR --output "/content/gdrive/MyDrive/"$PREFIX".pod5"

# Specify the accurate basecalling model - fast (fast), high (hac) or super (sup) accuracy
# dna_r9.4.1_e8_fast@v3.4
# dna_r9.4.1_e8_hac@v3.3
# dna_r9.4.1_e8_sup@v3.3
./ont-dorado/bin/dorado basecaller "ont-dorado/models/dna_r9.4.1_e8_sup@v3.3" \
"/content/gdrive/MyDrive/"$PREFIX".pod5" -r --min-qscore 8 --emit-fastq >> "/content/gdrive/MyDrive/"$PREFIX"_DORADO-BASECALL.fastq"

./ont-dorado/bin/dorado demux "/content/gdrive/MyDrive/"$PREFIX"_DORADO-SUP.fastq" --emit-fastq \
--output-dir "/content/gdrive/MyDrive/"$PREFIX"_DORADO-DEMUX" --kit-name $BARKIT -t $(nproc)